#####  DNN기반 회귀 모델 구현 + 학습 진행 모니터링 + 진행 파라미터 저장
- 데이터 셋 : iris.csv
- 피쳐/속성 : 3개 Sepal_Length, Sepal_Width, Petal_Length
- 타겟/라벨 : 1개 Petal_Width
- 학습-방법 : 지도학습 > 회귀
- 알고 리즘 : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성 
- 프레임워크 : Pytorch
***
- 모니터링
    * 기준 : 검증데이터셋의 loss 또는 score
    * 평가 : 학습데이터셋의 loss 또는 score와 비교해서 학습 중단여부 결정
    * 저장 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장


[1] 모듈 로딩 및 데이터 준비 <hr>

In [100]:

# 모듈 로딩
# - Model관련
import torch              
import torch.nn as nn              
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader 
import torch.optim as optim    
from torchmetrics.regression import R2Score, MeanSquaredError 
from torchinfo import summary 

#- Data 및 시각화 관련
import pandas as pd 
import matplotlib.pyplot as plt              
from sklearn.preprocessing import * 
from sklearn.model_selection import train_test_split  

In [101]:
# 활용 패키지 버전 체크 ==> 사용자 정의함수로 구현하세요~!!
print(f'Pytorch v.{torch.__version__}')
print(f'Pandas  v.{pd.__version__}')

Pytorch v.2.4.1
Pandas  v.2.0.3


In [102]:
###  데이터 로딩
DATA_FILE='../data/iris.csv' 

### CSV >>> DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

### 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스목적 : iris 데이터를 학습 및 추론 목적
- 클래스이름 : IrisRegModel
- 부모클래스 : nn.Module
- 매개__변수 : 층별 입출력 개수 고정하기때문에 필요 없음!
- 속성__필드 : 
- 기능__역할 : __init__() : 모델 구조 설정 ,  forward() : 순방향 학습 <= 오바라이딩(overriding)
- 클래스구조 
    * 입력층 : 입력 3개(피쳐)      출력   10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 : 입력 10개           출력   30개(퍼셉트론/뉴런 30개 존재)
    * 출력층 : 입력 30개           출력   1개(너비값)

- 활성화함수
    * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드 
    * 함수 형태 ==> torch.nn.functional 아래에  ==> forward()메서드

In [103]:
class IrisRegModel(nn.Module):
    
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(3, 10)
        self.hd_layer=nn.Linear(10, 30)
        self.out_layer=nn.Linear(30, 1)
        
    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        #- 입력층
        y=self.in_layer(input_data)   # fi1W11+f12W12+f13W13+b, ... , fi101W101+f102W102+fi103W103+b
        y=F.relu(y)                   # relu => y 값의 범위 : 0 <= y
        
        # - 은닉층 : 10개의 숫자 값(>=0)
        y=self.hd_layer(y)            # f21W11+f22W12...+f210W210+b, ... , f230W201+...+f230W210+b
        y=F.relu(y)                   # relu => y 값의 범위 : 0 <= y
        
        #- 출력층 : 30개의 숫자 값(>=0) 회귀이므로 바로 반환(return)
        return self.out_layer(y)              # f31W31+.....+f330W330+b  


In [104]:
### 모델 인스턴스 생성
model = IrisRegModel()

print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [105]:
# 모델 사용 메모리 정보 확인
summary(model, input_size=(1000000,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [1000000, 1]              --
├─Linear: 1-1                            [1000000, 10]             40
├─Linear: 1-2                            [1000000, 30]             330
├─Linear: 1-3                            [1000000, 1]              31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 401
Input size (MB): 12.00
Forward/backward pass size (MB): 328.00
Params size (MB): 0.00
Estimated Total Size (MB): 340.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터_셋 : iris.csv
- 피쳐_개수 : 3개
- 타겟_개수 : 1개
- 클래스이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성__필드 : featureDF, targetDF, n_rows, n_features
- 필수메서드 
    * _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * _ _len_ _(self) : 데이터의 개수 반환
    * _ _getItem_ _(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [106]:
class IrisDataset(Dataset):
    
    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF 
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values) 
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [107]:
## 데이터셋 인스턴스 생성

# - DataFram에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]   # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]]    # 2D (150, 1)

# - 커스텀데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF, targetDF)

[4] 학습 준비 
- 학습_횟수 : EPOCH         <- 처음~끝까지 공부하는 단위
- 배치_크기 : BATCH_SIZE    <- 한번에 학습할 데이터셋 양 
- 위치_지정 : DEVICE    <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학_습_률 : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1 

In [108]:

### 학습 진행 관련 설정
EPOPCH = 1
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]//BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

print(f'BATCH_CNT : {BATCH_CNT}')

BATCH_CNT : 15


- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수, 성능지표)

In [109]:

# 모델 인스턴스
model=IrisRegModel() 

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test =train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val =train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')

trainDS=IrisDataset(X_train, y_train)
valDS=IrisDataset(X_val, y_val)
testDS=IrisDataset(X_test, y_test)

# 데이터로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [110]:
## [테스트]데이터로더와 데이터셋 체크
for feature, target in trainDL: 
    print(feature.shape, target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [111]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE ....
reqLoss=nn.MSELoss()

[5] 학습 진행

- >>> 모델 저장 준비

In [112]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os


# 저장 경로
SAVE_PATH='../models/iris/'
# 저장 파일명
SAVE_FILE='model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명명
SAVE_MODEL='model_all.pth'

In [113]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)      # 폴더/폴더/... 하위 폴더까지 생성

- >>> 학습 진행

In [114]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY=[[],[]], [[],[]]
CNT=irisDS.n_rows/BATCH_SIZE
print(f'CNT => {CNT}')

## 학습 모니터링/스케줄링 설정 => LOSS_HISTORY, SCORE_HISTORY 활용
# => 임계기준 : 10번
BREAK_CNT=0
LIMIT_VALUE=10


for epoch in range(EPOPCH):
    # 학습 모드로 모델 설정
    model.train()
    
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0
    for featureTS, targetTS in trainDL:
        
        # 학습 진행
        pre_y=model(featureTS)
        
        # 손실 계산
        loss=reqLoss(pre_y, targetTS)
        loss_total += loss.item()
        
        # 성능평가 계산
        score=R2Score()(pre_y, targetTS)
        score_total += score.item()
        
        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featrueTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        # 추론/평가
        pre_val=model(val_featrueTS)
        # 손실
        loss_val=reqLoss(pre_val, val_targetTS)
        # 성능평가
        score_val=R2Score()(pre_val, val_targetTS)

        
    # 에포크 당 손실값과 성능평가값 저장    
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)
    
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # 학습 진행 모니터링/스케줄링 - 검증 DS 기준
    # Loss 기준
    if len(LOSS_HISTORY[1]) >= 2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]: BREAK_CNT +=1

    # Score 기준
    # if len(SCORE_HISTORY[1]) >= 2:
    #     if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]: BREAK_CNT +=1

    # 성능이 좋은 학습 가중치 저장
    SAVE_FILE=f'model_train_wbs_{epoch}_{score_val}.pth'
    if len(SCORE_HISTORY[1]) == 1:
        # 첫번째라서 무조건 모델 저장
        torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)
        # 모델 전체 저장
        torch.save(model, SAVE_PATH+SAVE_MODEL)
    else:
        if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]): 
            torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)
            torch.save(model, SAVE_PATH+SAVE_MODEL)

    # 학습 중단 여부 설정
    if BREAK_CNT > LIMIT_VALUE:
        print(f"성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단")
        break

CNT => 15.0


In [115]:
val_featrueTS=torch.FloatTensor(valDS.featureDF.values)
val_targetTS=torch.FloatTensor(valDS.targetDF.values)

val_featrueTS.shape, val_targetTS.shape

(torch.Size([28, 3]), torch.Size([28, 1]))

In [116]:
print(f'CNT => {CNT}')
print(f'LOSS_HISTORY => {LOSS_HISTORY}')
print(f'SCORE_HISTORY => {SCORE_HISTORY}')

CNT => 15.0
LOSS_HISTORY => [[1.0794759273529053], [tensor(1.3979)]]
SCORE_HISTORY => [[-1.7143045902252196], [tensor(-1.1656)]]


- 모델 저장 방법 <hr>
* 방법1 : 모델 파라미터만 저장
* 방법2 : 모델 설계 구조 및 파라미터까지 모두 저장

In [117]:
### 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[-0.1462,  0.1941,  0.0485],
                      [-0.5247,  0.4645,  0.0414],
                      [-0.3814, -0.3677, -0.5208],
                      [ 0.3986,  0.5283, -0.4023],
                      [-0.5405,  0.5249,  0.1623],
                      [ 0.1966, -0.5629, -0.2867],
                      [-0.0586,  0.2979, -0.3552],
                      [-0.4856,  0.5104, -0.5235],
                      [-0.5592, -0.4926,  0.3850],
                      [ 0.0592,  0.1808, -0.0944]])),
             ('in_layer.bias',
              tensor([-0.1613,  0.2998, -0.1063,  0.5346, -0.1362,  0.4976, -0.3429, -0.1709,
                       0.2848,  0.5264])),
             ('hd_layer.weight',
              tensor([[ 0.1952, -0.1906,  0.0870,  0.2190,  0.2959, -0.3026,  0.2439, -0.0675,
                        0.1638, -0.0022],
                      [ 0.2807, -0.0851,  0.0057,  0.0093,  0.0238, -0.0619,  0.1480,  0.3030,
                     

- [방법 1] 모델 파라미터 즉, 층별 가중치와 절편들

In [118]:
# 모델 저장
torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)

In [119]:
# 모델 즉, 가중치와 절편 로딩
# [1] 가중치와 절편 객체로 로딩
# [2] 모델의 state_dict 속성에 저장

# 읽기 
wbTS=torch.load(SAVE_PATH+SAVE_FILE, weights_only=True)

print(type(wbTS))

<class 'collections.OrderedDict'>


In [120]:
# 모델 인스턴스에 저장
model2=IrisRegModel()       # 층마다 W, b 초기화
model2.load_state_dict(wbTS)

<All keys matched successfully>

In [121]:
print(model2)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
